In [1]:
import numpy as np
import h5py
import os
import glob

from skimage import transform,color,io,exposure
from sklearn.model_selection import train_test_split

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD
from keras.layers import Dropout,Activation,Flatten,Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
IMG_SIZE = 40
SEED = 4
NUM_CLASSES = 43

Using Theano backend.
ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.8.0: cannot open shared object file: No such file or directory


In [2]:
def processimage(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])

In [20]:
root_dir = 'Training'
imgs = []
labels = []
all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
np.random.seed(SEED)
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = processimage(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)
X = np.array(imgs, dtype='float32')
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

In [21]:
def cnn_model():
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, IMG_SIZE, IMG_SIZE), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

model = cnn_model()
# let's train the model using SGD + momentum (how original).
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])


def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))

In [3]:
import pandas as pd
test = pd.read_csv('GT-final_test.csv',sep=';')

X_test = []
y_test = []
i = 0
for file_name, class_id  in zip(list(test['Filename']), list(test['ClassId'])):
    img_path = os.path.join('Testing',file_name)
    X_test.append(processimage(io.imread(img_path)))
    y_test.append(class_id)
    
X_test = np.array(X_test)
y_test = np.array(y_test)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

datagen.fit(X_train)

In [23]:
nb_epoch = 4
batch_size = 32
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val),
                            callbacks=[LearningRateScheduler(lr_schedule),
                                       ModelCheckpoint('model_aug.h5',save_best_only=True)]
                           )


Epoch 1/4
31367/31367 [==============================] - 164s - loss: 2.4708 - acc: 0.2816 - val_loss: 1.2652 - val_acc: 0.5814
Epoch 2/4
31367/31367 [==============================] - 167s - loss: 1.1811 - acc: 0.6129 - val_loss: 0.3965 - val_acc: 0.8843
Epoch 3/4
31367/31367 [==============================] - 162s - loss: 0.6904 - acc: 0.7754 - val_loss: 0.1916 - val_acc: 0.9392
Epoch 4/4
 5824/31367 [====>.........................] - ETA: 124s - loss: 0.5552 - acc: 0.8187

KeyboardInterrupt: 

In [24]:
modelf = load_model('model_aug.h5')
y_pred = modelf.predict_classes(X_test)
acc = float(np.sum(y_pred==y_test))/np.size(y_pred)
#a = precision(y_test, y_pred)
print("Test accuracy = {}".format(acc))

12630/12630 [==============================] - 19s    
Test accuracy = 0.899762470309


In [25]:
from sklearn import metrics

In [26]:
print(metrics.classification_report(y_test, y_pred, target_names=None,digits=6))

             precision    recall  f1-score   support

          0   1.000000  0.133333  0.235294        60
          1   0.837292  0.979167  0.902689       720
          2   0.906918  0.961333  0.933333       750
          3   0.969359  0.773333  0.860321       450
          4   0.968852  0.895455  0.930709       660
          5   0.784591  0.792063  0.788310       630
          6   0.990000  0.660000  0.792000       150
          7   0.670436  0.922222  0.776427       450
          8   0.858044  0.604444  0.709257       450
          9   1.000000  0.975000  0.987342       480
         10   1.000000  0.910606  0.953212       660
         11   0.942661  0.978571  0.960280       420
         12   0.938440  0.994203  0.965517       690
         13   0.994452  0.995833  0.995142       720
         14   1.000000  0.988889  0.994413       270
         15   0.990566  1.000000  0.995261       210
         16   0.920245  1.000000  0.958466       150
         17   1.000000  0.877778  0.934911   